In [23]:
#@ Importing necessary libraries and dependencies:
import torch
from torch import nn
from torch.utils.data import TensorDataset, Dataset, DataLoader
from torch.optim import SGD, Adam
import numpy
from torchvision import datasets

In [24]:
#@ For effective computation
device='cuda' if torch.cuda.is_available() else 'cpu'

In [25]:
#@ Creating dataset:
X_train=torch.tensor([[[[1, 2, 3, 4],
                        [2, 3, 4 ,5 ],
                        [5, 6, 7, 8],
                        [1, 3, 4, 5]]], [[[-1, 2, 3, -4],
                                          [2, -3, 4, 5],
                                          [-5, 6, -7, 8],
                                          [-1, -3, -4, -5]]]]).to(device).float()

In [26]:
X_train.shape  # batch_size=2, channel=1, height=4, width=4

torch.Size([2, 1, 4, 4])

In [28]:
X_train /= 8 #scaling values so that it lies between -1 to 1
             # dividing by the highest value
y_train=torch.tensor([0, 1]).to(device).float()
# X_train

In [12]:
#@ Model Building:
def ConvoNet():
  model=nn.Sequential(
         nn.Conv2d(1, 1, kernel_size=3), # 1 channel, 1 filter, size=3x3
         nn.MaxPool2d(2), #Maxpooling
         nn.ReLU(),
         nn.Flatten(),
         nn.Linear(1, 1),
         nn.Sigmoid(),
  ).to(device)

  loss_func=nn.BCELoss()
  optimizer=Adam(model.parameters(), lr=1e-3)
  return model, loss_func, optimizer

In [13]:
pip install torch_summary

In [14]:
from torchsummary import summary
model, loss_func, optimizer=ConvoNet()
summary(model, X_train);

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 1, 2, 2]             10
├─MaxPool2d: 1-2                         [-1, 1, 1, 1]             --
├─ReLU: 1-3                              [-1, 1, 1, 1]             --
├─Flatten: 1-4                           [-1, 1]                   --
├─Linear: 1-5                            [-1, 1]                   2
├─Sigmoid: 1-6                           [-1, 1]                   --
Total params: 12
Trainable params: 12
Non-trainable params: 0
Total mult-adds (M): 0.00
Input size (MB): 0.00
Forward/backward pass size (MB): 0.00
Params size (MB): 0.00
Estimated Total Size (MB): 0.00


In [29]:
def train_batch(x, y, model, optimizer, loss_func):
  model.train()
  prediction=model(x) #forward pass
  batch_loss=loss_func(prediction.squeeze(0), y) #squeeze removes first dim
                                                # for (1, n), (n,)
  batch_loss.backward() #backpropagation
  optimizer.step()
  optimizer.zero_grad()
  return batch_loss.item()

In [30]:
training_dataloader=DataLoader(TensorDataset(X_train, y_train))

In [21]:
for epoch in range(2000):
  for ix, batch in enumerate(iter(training_dataloader)):
    x, y=batch
    batch_loss=train_batch(x, y, model, optimizer, loss_func)

In [22]:
model(X_train[:1])

tensor([[0.1246]], device='cuda:0', grad_fn=<SigmoidBackward0>)